In [1]:
from main import ee, feats, l8sr
print(feats.keys())

dict_keys(['Tailing', 'Processing_Facilities', 'Dumps', 'Active_Mines', 'Artisinal_Mining', 'Smelters', 'Exploration'])


In [2]:
import ipyleaflet
from ipywidgets import HTML

# Setup interactive map
ee_map = ipyleaflet.Map(zoom=3, layout={'height':'600px'}, scroll_wheel_zoom=True)

def GetTileLayerUrl(ee_image_object):
    """Load tile url from EE for a given image object"""
    map_id = ee.Image(ee_image_object).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)

def ResetLayers(map_object):
    """Clear map overlays"""
    for lyr in map_object.layers[1:]:
        map_object.remove_layer(lyr)

def AddTileLayer(map_object, ee_image_object):
    """Add EE tile overlay"""
    tile_url = GetTileLayerUrl(ee_image_object)
    map_object.add_layer(ipyleaflet.TileLayer(url=tile_url))

def AddPointsLayer(map_object, feats):
    """Add point features overlay"""
    markers = []
    geojson = feats.getInfo()
    for feat in geojson['features']:
        geo = feat['geometry']
        if geo['type'] != 'Point': continue
        msg = HTML()
        msg.value = feat['properties']['name']
        msg.description = feat['properties']['description']
        lng, lat = geo['coordinates']
        marker = ipyleaflet.Marker(location=(lat, lng), draggable=False)
        marker.popup = msg
        markers.append(marker)
    marker_cluster = ipyleaflet.MarkerCluster(
        markers=markers
    )
    map_object.add_layer(marker_cluster) 

In [3]:
# Visualize points
ResetLayers(ee_map)
filtered = (
    l8sr.filterDate('2016-01-01', '2017-01-01')
        .median() # or mean, max, min
    
        # min=value to map to RGB8 value 0
        # max=value to map to RGB8 value 255
        .visualize(min=0, max=3000, bands=['red', 'green', 'blue'])
)
AddTileLayer(ee_map, filtered)
AddPointsLayer(ee_map, feats['Active_Mines'])
ee_map

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

# References

- <https://github.com/tylere/PyDataNYC2017/blob/master/ipynb/satellite_analysis.ipynb>
- <https://github.com/tylere/ee-jupyter-examples/blob/master/2%20-%20EE%20101.ipynb>
- <https://ipyleaflet.readthedocs.io/en/latest/api_reference/map.html>
- <https://github.com/tylere/ee-jupyter-examples/blob/master/4%20-%20Earth%20Engine%20and%20TensorFlow.ipynb>
- <https://developers.google.com/earth-engine/api_docs>